# Demonstration of the implementation concept of CDT

This notebook accompanies the paper describing the 

Automated detection of propagating cracks in RC beams based on DIC driven modeling of damage localization,
F. Seemab, M. Schmidt, A. Baktheer, M. Classen, R. Chudoba, Engineering Structures, 2023

that presents the implementation concept that exploting the close relation between the mathematical formulation using the indexed-based notation based on Eistein summation rule and the direct execution using the `einsum` method provied in the `numpy` package for multi-dimensional arrays. 

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pylab as plt
import collections
collections.Iterable = collections.abc.Iterable

In [ ]:
from scipy.interpolate import \
  RegularGridInterpolator as RGI

In [ ]:
test = 'B9_TV2'

In [ ]:
from bmcs_shear.api import DICGridTri
dic_grid = DICGridTri(U_factor=100, dir_name=test, t=1, padding=40, 
                      d_x=3, d_y=3, n_T_max=40, T_stepping='delta_T')
dic_grid.read_beam_design()
time_T, F_T = dic_grid.time_F_T
w_T = dic_grid.w_T
np.savez_compressed(test, X_TPa=dic_grid.X_TQa[:,:,:-1], F_T=F_T, w_T=w_T)

## Input format

In [ ]:
loaded = np.load('{}.npz'.format(test))
X_TPa, F_T, w_T = loaded['X_TPa'], loaded['F_T'], loaded['w_T']

In [ ]:
X_0Pa = X_TPa[0]
U_TPa = X_TPa - X_0Pa[None,...]
n_T = len(U_TPa)

## Grid definition

In [ ]:
pad_a = np.array([40, 40])
X_min_a = np.array([np.min(X_0Pa[:,a]+pad_a[a]) for a in (0,1)]) # left&bottom
X_max_a = np.array([np.max(X_0Pa[:,a]-pad_a[a]) for a in (0,1)]) # right&top

In [ ]:
L_a = X_max_a - X_min_a # frame dimensions
d = 4 # 4 mm distances
n_a = 2 # number of spatial dimensions
n_I, n_J = np.array( L_a / d, dtype=np.int_ )
d_X_a = [L_a[0]/n_I, L_a[1]/n_J]
n_I, n_J

Given a column index $I = [0, n_I-1\}$, row index $J = [0, n_J-1]$ and direction index $a = [0,1]$ 
the index expression
$$
\mathcal{G}_{IJa} = (1-a)I + a J
$$
introduces a grid index map rendering the horizontal/column indexes of individual grid nodes for $a=0$ and vertical / row indexes for $a=1$.

$$
 a = 0 \rightarrow (\zeta = 1, \eta = 0)
$$
$$
 a = 1 \rightarrow (\zeta = 0, \eta = 1)
$$
$$
\zeta = 1-a
$$
$$
\eta = a
$$

In [ ]:
I,J,a = [np.arange(n) for n in (n_I,n_J,n_a)]
G_aIJ = (np.einsum('a, I->aI', (1-a), I)[:,:, None] + 
         np.einsum('a, J->aJ', a, J)[:,None, :])

Given the step length 
$$
\Delta X_a = \left[\frac{L_x}{n_I}, \frac{L_y}{n_J} \right],
$$
the distance between grid points is
The coordinates of all nodes are expressed as
$$
X_{aIJ} = \Delta X_{a} \, \mathcal{G}_{aIJ}
$$

In [ ]:
X_aIJ = X_min_a[:,None,None] + np.einsum('aIJ,a->aIJ', G_aIJ, d_X_a)
X_IJa = np.einsum('aIJ->IJa', X_aIJ)

In [ ]:
from scipy.spatial import Delaunay
from scipy.interpolate import \
    LinearNDInterpolator as LNDI
tri = Delaunay(X_0Pa)

Use an interpolator over the Delaunay triangulation to obtain the values over a regular grid $X_{aIJ}$

In [ ]:
U_TIJa = np.array([
    LNDI(tri, U_TPa[T])(*X_aIJ) for T in range(n_T)
])

The enumeration of nodes within a single element is defined for the $\xi$ and $\eta$ directions consistently with the enumeration of the $\mathcal{G}_{aIJ}$ grid by setting 
$$
{g}_{aij} = \mathcal{G}_{a; I=i; J=i}, \; i \in (0,1) \times (0,1)
$$

In [ ]:
n_i, n_j = 2, 2
g_aij = G_aIJ[:,:n_i,:n_j]

By introducing the element indexes $E = [0, n_I-2]$ and $F = [0, n_J-2]$ in the 
horizontal and vertical direction, we can introduce the index map identifying
the local element nodes enumerated counter clock-wise in each element of the grid as
$$
\mathcal{H}_{aEiFj} = \mathcal{G}_{aEF} + g_{aij}
$$

In [ ]:
n_E, n_F = n_I-1, n_J-1
G_aEF = G_aIJ[:,:-1,:-1]

In [ ]:
H_aEiFj = G_aEF[:,:,None,:,None] + g_aij[:,None,:,None,:]

$$
  X_{EiFja} = X_{I=\mathcal{H}_{0EiFj}, I=\mathcal{H}_{1EiFj}}
$$

In [ ]:
X_EiFja = X_IJa[(*H_aEiFj,)]
U_TEiFja = U_TIJa[(slice(None), *H_aEiFj)]

## Nodal coordinates and quadrature points of an element

In [ ]:
delta_rs = np.eye(2, dtype=np.int_)

$$
\xi_{rij} = 2 X_{a;E=0;i;F=0;j} - 1
$$

In [ ]:
xi_rij = (H_aEiFj[:,0,:,0,:] * 2) - 1

$$
 \eta_{rmn} = \frac{1}{\sqrt{3}} \xi_{r;i=m;j=n}, \; m, n \in (0,1) \times (0,1)
$$

In [ ]:
n_m, n_n = n_i, n_j
eta_rmn = 3**(-1/2) * xi_rij

## Bilinear Lagrange shape functions

Dimensional directions explicitly referenced in the product expression
$$
N_{ijmn} = \frac{1}{4}(1 + \eta_{r=0;mn} \xi_{r=0;ij})\,(1 + \eta_{r=1;mn} \xi_{r=1;ij})
$$

In [ ]:
N1_ijmn = (
    (1 + np.einsum('mn,ij->mnij', eta_rmn[0], xi_rij[0]))* 
    (1 + np.einsum('mn,ij->mnij', eta_rmn[1], xi_rij[1]))
) / 4

Dimensional directions included in the index operator
$$
N_{ij}(\eta_r) 
=
\frac{1}{4}(
1 + \eta_0 \xi_{0ij} + \eta_1 \xi_{1ij} + \eta_0 \xi_{0ij} \eta_1 \xi_{1ij}
)
$$

$$
N_{ijmn}
=
\frac{1}{4}\left(
1 + \eta_{rmn} \xi_{rij} + \frac{1}{2}(1 - \delta_{rs}) \eta_{smn} \xi_{sij} \eta_{rmn} \xi_{rij}
\right)
$$

In [ ]:
N_ijmn = (1 + 
  np.einsum('rmn,rij->ijmn', eta_rmn, xi_rij) +
  np.einsum('rs,smn,sij,rmn,rij->ijmn', (1-delta_rs), eta_rmn, xi_rij, eta_rmn, xi_rij) / 2
)/4
np.sum(N_ijmn - N1_ijmn)

Derivatives of the shape functions w.r.t. parametric coordinates

$$
\frac{\partial N_{ij} }{\partial \eta_0}
= 
\frac{1}{4}( \xi_{0ij} + \eta_1 \xi_{1ij} ), \;\;
\frac{\partial N_{ij} }{\partial \eta_1}
= 
\frac{1}{4}( \xi_{1ij} + \eta_0 \xi_{0ij} )
$$

can be rewritten in the index notation as

$$
N_{ij,s}(\eta_r)
= 
\frac{1}{4}( \xi_{sij} +
(1-\delta_{rs}) \xi_{sij} \eta_{r} \xi_{rij}
)
$$

and directly evaluated in the quadrature points $\eta_{rmn}$

$$
N_{ijmn,s}
= 
\frac{1}{4}
\left[ \xi_{sij} +
(1-\delta_{rs}) \, \xi_{sij} \eta_{rmn} \xi_{rij}
\right]
$$

In [ ]:
dN_sijmn = (
    xi_rij[:,:,:,None,None] + 
    np.einsum('rs,sij,rmn,rij->sijmn', (1 - delta_rs), xi_rij, eta_rmn, xi_rij)
) / 4

## Kinematic operator

Jacobi matrix in every quadrature point reads
$$
J_{EmFnas} = N_{ijmn,s} X_{EiFja}
$$

In [ ]:
J_EmFnas = np.einsum(
 'sijmn,EiFja->EmFnas',
 dN_sijmn, X_EiFja
)

The inverse of Jacobi matrix in all quadrature points

In [ ]:
inv_J_EmFnsa = np.linalg.inv(J_EmFnas)

The strain operator 
$$
 \varepsilon_{ab} = \frac{1}{2}( u_{a,b} + u_{b,a} )
$$
is represented by the cross combination of the dimensions as

In [ ]:
delta_ab = np.eye(2)
Diff1_abcd = 0.5 * (
    np.einsum('ac,bd->abcd', delta_ab, delta_ab) +
    np.einsum('ad,bc->abcd', delta_ab, delta_ab)
)

$$
B_{EiFjmnabc} = D_{abcd} N_{ijmn,s} J^{-1}_{EmFnsd}
$$

In [ ]:
B_EiFjmnabc = np.einsum(
    'abcd,sijmn,EmFnsd->EiFjmnabc',
    Diff1_abcd, dN_sijmn, inv_J_EmFnsa
)

$$
\varepsilon_{EmFnab} = B_{EiFjmnabc} U_{EiFjc}
$$

In [ ]:
eps_TEmFnab = np.einsum(
    'EiFjmnabc,TEiFjc->TEmFnab',
    B_EiFjmnabc, U_TEiFja
)
eps_TEmFnab.shape

## Scalar damage model

The strain state is reduced to a scalar value using maximum value of the principal strain
$\kappa_{TEmFn}$ and reshaped to a grid $K \times L$.

In [ ]:
eps_TEmFna, _ = np.linalg.eig(eps_TEmFnab)
kappa_TEmFn = np.max(eps_TEmFna, axis=-1)
kappa_TKL = kappa_TEmFn.reshape(-1, n_E*n_m, n_F*n_n)

Then, the damage value is quantified using the exponential damage law
$$
\omega = \frac{\varepsilon_\mathrm{o}}{\kappa} \exp\left(-\frac{\kappa - \varepsilon_\mathrm{o}}
{ \varepsilon_\mathrm{f} - \varepsilon_\mathrm{o} }\right)
$$

In [ ]:
E_ = 1400
nu_ = 0.18
eps_0=0.002
#eps_f=0.0028
eps_f=0.005
I = np.where(kappa_TEmFn>=eps_0)
omega_TEmFn = np.zeros_like(kappa_TEmFn)
omega_TEmFn[I] = 1.0-(eps_0/kappa_TEmFn[I]*np.exp(
    -(kappa_TEmFn[I]-eps_0)/(eps_f-eps_0))
)

## Plot localized damage field

In [ ]:
X_aEmFn = np.einsum('ijmn,EiFja->aEmFn', N_ijmn, X_EiFja)
X_aKL = X_aEmFn.reshape(-1,(n_I-1)*2, (n_J-1)*2)

In [ ]:
omega_TKL = omega_TEmFn.reshape(-1, n_E*n_m, n_F*n_n)

In [ ]:
fig = plt.figure(figsize=(12,5))
rows = fig.subplots(2,2)
for (ax_s, ax_o), T in zip(rows, [10, -1]):
    ax_o.contourf( X_aKL[0], X_aKL[1], omega_TKL[T], cmap='BuPu',vmin=0, vmax=1)
    ax_o.axis('equal');
    ax_o.axis('off');
    ax_s.contourf( X_aKL[0], X_aKL[1], kappa_TKL[T], cmap='BuPu', vmin=eps_0, vmax=50*eps_0)
    ax_s.axis('equal');
    ax_s.axis('off');

## Constitutive law

In [ ]:
la = E_*nu_/((1+ nu_)*(1-2*nu_))
mu = E_/(2+2*nu_)
delta = np.eye(2)
D_abef = (
 np.einsum(',ij,kl->ijkl',la,delta,delta)+
 np.einsum(',ik,jl->ijkl',mu,delta,delta)+
 np.einsum(',il,jk->ijkl',mu,delta,delta))

In [ ]:
E = E_
nu = nu_
D_stress = np.zeros([3, 3])
D_stress[0, 0] = E / (1 - nu * nu)
D_stress[0, 1] = E / (1 - nu * nu) * nu
D_stress[1, 0] = E / (1 - nu * nu) * nu
D_stress[1, 1] = E / (1 - nu * nu)
D_stress[2, 2] = E / (1 - nu * nu) * (1 / 2 - nu / 2)

In [ ]:
D_strain = np.zeros([3, 3])
D_strain[0, 0] = E * (1.0 - nu) / (1.0 + nu) / (1.0 - 2.0 * nu)
D_strain[0, 1] = E / (1.0 + nu) / (1.0 - 2.0 * nu) * nu
D_strain[1, 0] = E / (1.0 + nu) / (1.0 - 2.0 * nu) * nu
D_strain[1, 1] = E * (1.0 - nu) / (1.0 + nu) / (1.0 - 2.0 * nu)
D_strain[2, 2] = E * (1.0 - nu) / (1.0 + nu) / (2.0 - 2.0 * nu)

In [ ]:
map2d_ijkl2a = np.array([[[[0, 0],
                              [0, 0]],
                             [[2, 2],
                              [2, 2]]],
                            [[[2, 2],
                              [2, 2]],
                             [[1, 1],
                              [1, 1]]]],
                       dtype=np.int_)
map2d_ijkl2b = np.array([[[[0, 2],
                              [2, 1]],
                             [[0, 2],
                              [2, 1]]],
                            [[[0, 2],
                              [2, 1]],
                             [[0, 2],
                              [2, 1]]]],
                       dtype=np.int_)


In [ ]:
D_abcd = D_stress[map2d_ijkl2a, map2d_ijkl2b]

In [ ]:
sig_TEmFnab = np.einsum('...,abef,...ef -> ...ab', 
 (1-omega_TEmFn), D_abef, eps_TEmFnab)
sig_TEmFnab2 = np.einsum('...,abef,...ef -> ...ab', 
 (1-omega_TEmFn), D_abcd, eps_TEmFnab)

In [ ]:
sig_TEmFna, _ = np.linalg.eig(sig_TEmFnab)
sig_TEmFna2, _ = np.linalg.eig(sig_TEmFnab2)

In [ ]:
max_sig = np.max(sig_TEmFna)
max_sig2 = np.max(sig_TEmFna2)
D_sig = max_sig - max_sig2
max_sig, max_sig2, D_sig, max_sig-max_sig2

In [ ]:
eps_T, sig_T = np.max(eps_TEmFna[-1], axis=-1).flatten(), np.max(sig_TEmFna[-1], axis=-1).flatten()
_, ax = plt.subplots(1,1)
ax.plot(eps_T, sig_T,'.')
ax.plot([0, 0.0005], [0, 3], color='red')

In [ ]:
sig_TKL = np.max(sig_TEmFna2, axis=-1).reshape(-1, n_E*n_m, n_F*n_n)
sig_TKL[sig_TKL<0] = 0

In [ ]:
from matplotlib import cm
fig = plt.figure(figsize=(11,3), tight_layout=True)
ax_rows = fig.subplots(3,2)
for (ax_o, ax_s), T in zip(ax_rows, [3, 8, -1]):
    # ax_e.contourf( X_aKL[0], X_aKL[1], kappa_TKL[T]-eps_0, cmap='BuPu', vmin=0, vmax=eps_0*3)
    # ax_e.axis('equal');
    # ax_e.axis('off');
    ax_o.contourf( X_aKL[0], X_aKL[1], omega_TKL[T], cmap='BuPu', vmin=0, vmax=1)
    ax_o.axis('equal');
    ax_o.axis('off');
    cs_sig = ax_s.contourf( X_aKL[0], X_aKL[1], sig_TKL[T], cmap='RdPu', vmin=0.1)
    ax_s.axis('equal');
    ax_s.axis('off');
# cbar_sig = fig.colorbar(cm.ScalarMappable(norm=cs_sig.norm, cmap=cs_sig.cmap),
#                         ax=ax_s, #ticks=np.arange(0, max_fe_eps * 1.01, 0.005),
#                         orientation='horizontal')    

In [ ]:
I1_TEmFn2 = np.einsum('aa,...aa->...', delta_ab, sig_TEmFnab2)
s_TEmFnab2 = sig_TEmFnab2 - np.einsum('ab,...->...ab', delta_ab, I1_TEmFn2) 
J2_TEnFn2 = 0.5*np.einsum('...ab,...ba->...', s_TEmFnab2, s_TEmFnab2)

In [ ]:
I1_TKL = I1_TEmFn2.reshape(-1, n_E*n_m, n_F*n_n)
J2_TKL = J2_TEnFn2.reshape(-1, n_E*n_m, n_F*n_n)

In [ ]:
from matplotlib import cm
fig = plt.figure(figsize=(11,3))
ax_rows = fig.subplots(3,3)
print(ax_rows)
for (ax_comp, ax_o, ax_s), T in zip(ax_rows, [1, 10, -1]):
    I1_KL = np.copy(I1_TKL[T])
    I1_KL[I1_KL > 0] = 0
    vmin, vmax= -8, -0.1
    levels = np.linspace(vmin, vmax, 4)
    cs_comp = ax_comp.contourf( X_aKL[0], X_aKL[1], I1_KL, levels=levels, vmin=vmin, vmax=vmax) #, cmap='RdPu')
    ax_comp.axis('equal');
    ax_comp.axis('off');
    vmin, vmax = 1, 6
    levels = np.linspace(vmin, vmax, 6)
    ax_o.contourf( X_aKL[0], X_aKL[1], I1_TKL[T], levels=levels, vmin=vmin, vmax=vmax, cmap='RdPu')
    ax_o.axis('equal');
    ax_o.axis('off');
    s = slice(0,None)
    cs_sig = ax_s.contourf( X_aKL[0][s], X_aKL[1][s], J2_TKL[T][s], levels=levels, vmin=vmin, vmax=vmax, cmap='RdPu')
    ax_s.axis('equal');
    ax_s.axis('off');
fig.subplots_adjust(right=0.92)
cbar_ax = fig.add_axes([0.95, 0.1, 0.01, 0.8])
fig.colorbar(cs_sig, cax=cbar_ax)

fig.subplots_adjust(left=0.08)
cbar_ax = fig.add_axes([0.05, 0.1, 0.01, 0.8])
fig.colorbar(cs_comp, cax=cbar_ax)

# fig.colorbar(cs_sig) # cm.ScalarMappable(norm=cs_sig.norm, cmap=cs_sig.cmap),
# cbar_sig = fig.colorbar(cm.ScalarMappable(norm=cs_sig.norm, cmap=cs_sig.cmap),
#                          ax=ax_s, #ticks=np.arange(0, max_fe_eps * 1.01, 0.005),
#                          orientation='horizontal')    

## Smoothing and interpolation of the damage field

In [ ]:
from scipy.interpolate import RectBivariateSpline

In [ ]:
x_K, y_L = X_aKL[0,:,0], X_aKL[1,0,:]

In [ ]:
get_omega_xy = RectBivariateSpline(x_K, y_L, omega_TKL[-1])

In [ ]:
omega_ipl_MN = get_omega_xy(x_K[:,None], y_L[None,:])

In [ ]:
np.max(omega_ipl_MN)

In [ ]:
fig = plt.figure(figsize=(8,5))
ax1, ax2 = fig.subplots(2,1)
for ax, T in zip([ax1, ax2], [10, -1]):
    get_omega_xy = RectBivariateSpline(x_K, y_L, omega_TKL[T], s=40)
    omega_ipl_MN = get_omega_xy(x_K[:,None], y_L[None,:])
    ax.contourf( X_aKL[0], X_aKL[1], omega_ipl_MN, cmap='BuPu',
                levels=[0.0, 0.2, 0.4, 0.8, 0.9, 1.0], vmin=0, vmax=1)
    ax.axis('equal');
    ax.axis('off');

In [ ]:
la_T = F_T / np.max(F_T)

In [ ]:
x_I, y_J = X_aIJ[0,:,0], X_aIJ[1,0,:]
X_TIJa = X_IJa[None,...] + U_TIJa
get_omega_lxy = RGI((la_T, x_K, y_L), omega_TKL)
get_Xa_lxy = RGI((la_T, x_I, y_J), X_TIJa) 
get_Ua_lxy = RGI((la_T, x_I, y_J), U_TIJa) 

In [ ]:
get_Xa_lxy([1, 0.200, 100])